In [ ]:
# Install ALL system dependencies (including petsc, slepc, boost, and eigen)
!apt-get -qq install -y build-essential gfortran openmpi-bin libopenmpi-dev python python-dev python-pip git python-tk libpetsc-complex-3.7 libpetsc-complex-3.7-dev libslepc-complex-3.7 libslepc-complex-3.7-dev libboost-all-dev libeigen3-dev 

# Install some of the optional python requirements
!pip install requests matplotlib numpy scipy mpi4py h5py

# Clone the emopt repo
!git clone https://github.com/anstmichaels/emopt.git

# Install emopt
!cd emopt && git checkout colab && PETSC_DIR='/usr/lib/petsc' PETSC_ARCH='' SLEPC_DIR='/usr/lib/slepc' python3 setup.py install

# Restart the runtime -- necessary to make the newly installed modules visible to the interpreter
exit()

In [ ]:
%matplotlib inline
"""This example is adopted from examples/simple_waveguide/simple_waveguide.py
"""

import emopt
from emopt.misc import NOT_PARALLEL

import numpy as np

####################################################################################
#Simulation Region parameters
####################################################################################
X = 10.0
Y = 7.0
dx = 0.02
dy = 0.02
wlen = 1.55
sim = emopt.fdfd.FDFD_TE(X, Y, dx, dy, wlen)

# by default, PML size is chosen for you. If you want to specify your own PML
# sizes you can set them using the sim.w_pml attribute which is an array with 4
# values [w_xmin, w_xmax, w_ymin, w_ymax] where each value is a PML width for
# the corresponding simulation boundary. e.g.
# sim.w_pml = [dx*12, dx*12, dx*12, dx*12]

# Get the actual width and height
# The true width/height will not necessarily match what we used when
# initializing the solver. This is the case when the width is not an integer
# multiple of the grid spacing used.
X = sim.X
Y = sim.Y
M = sim.M
N = sim.N


####################################################################################
# Setup system materials
####################################################################################
# Materials
n0 = 1.0
n1 = 3.0

# set a background permittivity of 1
eps_background = emopt.grid.Rectangle(X/2, Y/2, 2*X, Y)
eps_background.layer = 2
eps_background.material_value = n0**2

# Create a high index waveguide through the center of the simulation
h_wg = 0.5
waveguide = emopt.grid.Rectangle(X/2, Y/2, 2*X, h_wg)
waveguide.layer = 1
waveguide.material_value = n1**2

eps = emopt.grid.StructuredMaterial2D(X, Y, dx, dy)
eps.add_primitive(waveguide)
eps.add_primitive(eps_background)

mu = emopt.grid.ConstantMaterial2D(1.0)

# set the materials used for simulation
sim.set_materials(eps, mu)

####################################################################################
# setup the sources
####################################################################################
# setup the sources -- just a dipole in the center of the waveguide
Jz = np.zeros([M,N], dtype=np.complex128)
Mx = np.zeros([M,N], dtype=np.complex128)
My = np.zeros([M,N], dtype=np.complex128)
Jz[M//2, N//2] = 1.0

sim.set_sources((Jz, Mx, My))

####################################################################################
# Build and simulate
####################################################################################
sim.build()
sim.solve_forward()

# Get the fields we just solved for
# define a plane using a DomainCoordinates with no z-thickness
sim_area = emopt.misc.DomainCoordinates(1.0, X-1.0, 1.0, Y-1.0, 0, 0, dx, dy, 1.0)
Ez = sim.get_field_interp('Ez', sim_area)

# Simulate the field.  Since we are running this using MPI, we only generate
# plots in the master process (otherwise we would end up with a bunch of
# plots...). This is accomplished using the NOT_PARALLEL flag which is defined
# by emopt.misc
if(NOT_PARALLEL):
    import matplotlib.pyplot as plt

    extent = sim_area.get_bounding_box()[0:4]

    f = plt.figure()
    ax = f.add_subplot(111)
    im = ax.imshow(Ez.real, extent=extent,
                            vmin=-np.max(Ez.real)/1.0,
                            vmax=np.max(Ez.real)/1.0,
                            cmap='seismic')

    # Plot the waveguide boundaries
    ax.plot(extent[0:2], [Y/2-h_wg/2, Y/2-h_wg/2], 'k-')
    ax.plot(extent[0:2], [Y/2+h_wg/2, Y/2+h_wg/2], 'k-')

    ax.set_title('E$_z$', fontsize=18)
    ax.set_xlabel('x [um]', fontsize=14)
    ax.set_ylabel('y [um]', fontsize=14)
    f.colorbar(im)
    plt.show()
